In [145]:
#fetches divvy info for a given lat/lon
#https://gbfs.divvybikes.com/gbfs/gbfs.json

In [1]:
import urllib.request, json, datetime, time, math, os
from operator import itemgetter

In [2]:
def calcdistance(pos1,pos2):
    R = 6373.0
    lat1 = math.radians(pos1[0])
    lon1 = math.radians(pos1[1])
    lat2 = math.radians(pos2[0])
    lon2 = math.radians(pos2[1])
    dlon = lon2-lon1
    dlat = lat2-lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance #returns distance in km between two lat/lon points

In [3]:
def nearbystations(lat,lon,maxdist):
    with urllib.request.urlopen("https://gbfs.divvybikes.com/gbfs/en/station_information.json") as url:
        stations = json.loads(url.read().decode())
    with urllib.request.urlopen("https://gbfs.divvybikes.com/gbfs/en/station_status.json") as url:
        station_status = json.loads(url.read().decode())
    station_data = []
    for station in stations["data"]["stations"]:
        station_lat = station["lat"]
        station_lon = station["lon"]
        distance = calcdistance([station["lat"],station["lon"]],[lat,lon])
        if distance < maxdist:
            name = station["name"]
            suffixes = [" Ave "," Rd "," St "," Pl "," Blvd "," Ct "," Pkwy "," Dr ", " St "]
            #^ removes end of street names for brevity
            for item in suffixes:
                name = name.replace(item," ")
            each_station = {
                "id":station["legacy_id"],
                "name":name,
                "distance":distance
            }
            for stationstatus in station_status["data"]["stations"]:
                if each_station["id"] == stationstatus["legacy_id"]:
                    each_station["bikes"] = stationstatus["num_bikes_available"] - stationstatus["num_ebikes_available"]
                    each_station["docks"] = stationstatus["num_docks_available"]
                    each_station["ebikes"] = stationstatus["num_ebikes_available"]
            station_data.append(each_station)
    return sorted(station_data, key = lambda i: i['distance'])
    #returns a dict with info about each station within maxdist of the given location, sorted from closest to furthest.

In [4]:
def nearbybikes(lat,lon,maxdist):
    with urllib.request.urlopen("https://gbfs.divvybikes.com/gbfs/en/free_bike_status.json") as url:
        freebikes = json.loads(url.read().decode())
    nearbikes = []
    for bike in freebikes["data"]["bikes"]:
        distance = calcdistance([bike["lat"],bike["lon"]],[lat,lon])
        if distance < maxdist and bike["is_reserved"]==0 and bike["is_disabled"]==0:
            nearbikes.append(distance)
    nearbikes.sort()
    return nearbikes[0:4]
    #returns up to 5 dockless bikes within maxdist of the given location, sorted from closest to furthest.
    #the format of the output is a simple list of distances in km of these bikes.

In [5]:
#this function combines the info for divvy stations with dockless ebikes into easily human-readable form
#dockless bikes are returned only if they are closer than the closest divvy station by default
#see below for what to change to return all dockless bikes in the desired radius

#input is with lat/lon as floats, and a float for distance in km

def divvyfinder(lat,lon,dist):
    stationdata = nearbystations(lat,lon,dist)
    closest_sta_distance = dist
    output = []
    for station in stationdata:
        stationprint = station["name"]+": "
        if station["bikes"] == 1:
            stationprint += "1 bike, "
        else:
            stationprint += str(station["bikes"])+" bikes, "
        if station["ebikes"] > 1:
            stationprint += str(station["ebikes"])+" ebikes, "
        elif station["ebikes"]==1:
            stationprint += "1 ebike, "
        if station["docks"]==1:
            stationprint += "1 dock"
        else:
            stationprint += str(station["docks"])+" docks"
        output.append(stationprint)
        if station["distance"]<closest_sta_distance and station["bikes"]+station["ebikes"]>0:
            closest_sta_distance = station["distance"]
    output = output[0:5]
    
    #for dockless bikes closer than the cloest station:
    ebikedata = nearbybikes(lat,lon,closest_sta_distance)
    #for dockless bikes within the dist of the given location:
    #ebikedata = nearbybikes(lat,lon,dist)
    
    if len(ebikedata)==1:
        freebikeprint = "Available E-bike "+str(round(ebikedata[0]*1000))+" meters away"
    elif len(ebikedata)==0:
        freebikeprint = None
    else:
        freebikeprint = "Available E-bikes "
        for bike in ebikedata:
            freebikeprint += str(round(bike*1000))+", "
        freebikeprint = freebikeprint[:-2] + " meters away"
    if freebikeprint != None:
        output.append(freebikeprint)
    return output
    #output format is a list
    #first are stations, each has a string with name of the station, number of bikes, number of ebikes, and number of docks
    #then a string with distances of ebikes

In [6]:
divvyfinder(41.882, -87.6278, 1)

['Dearborn & Monroe St: 14 bikes, 3 ebikes, 19 docks',
 'Michigan & Madison St: 0 bikes, 1 ebike, 16 docks',
 'Daley Center Plaza: 15 bikes, 7 ebikes, 17 docks',
 'State & Randolph St: 0 bikes, 9 docks',
 'Millennium Park: 1 bike, 1 ebike, 39 docks',
 'Available E-bike 106 meters away']

In [8]:
nearbystations(41.954,-87.652,3)

[{'id': '240',
  'name': 'Sheridan & Irving Park Rd',
  'distance': 0.2008800158458249,
  'bikes': 6,
  'docks': 19,
  'ebikes': 1},
 {'id': '256',
  'name': 'Broadway & Sheridan Rd',
  'distance': 0.21074144177539506,
  'bikes': 10,
  'docks': 0,
  'ebikes': 3},
 {'id': '254',
  'name': 'Pine Grove & Irving Park Rd',
  'distance': 0.3300811206074064,
  'bikes': 2,
  'docks': 9,
  'ebikes': 3},
 {'id': '312',
  'name': 'Clarendon & Gordon Ter',
  'distance': 0.4770222974739678,
  'bikes': 3,
  'docks': 12,
  'ebikes': 0},
 {'id': '114',
  'name': 'Sheffield & Waveland Ave',
  'distance': 0.5528778067749528,
  'bikes': 8,
  'docks': 17,
  'ebikes': 1},
 {'id': '306',
  'name': 'Sheridan & Buena Ave',
  'distance': 0.5568280441495999,
  'bikes': 2,
  'docks': 15,
  'ebikes': 2},
 {'id': '304',
  'name': 'Broadway & Waveland Ave',
  'distance': 0.6145354228066932,
  'bikes': 8,
  'docks': 5,
  'ebikes': 0},
 {'id': '232',
  'name': 'Pine Grove & Waveland Ave',
  'distance': 0.681285272833